In [ ]:
try:
    spark.stop()
except:
    pass

# Using findspark to find automatically the spark folder
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [ ]:
spark = SparkSession.builder.appName("dieseaseStream").getOrCreate()

In [ ]:
line = spark.readStream.format("socket").option("host", "localhost").option("port", 9998).load()

In [ ]:
point = line.select(explode(split(line.value," ")).alias("lon"))
point = point.withColumn('lat', split(point["lon"], '\,')[1])
point = point.withColumn('lon', split(point["lon"], '\,')[0])
pointCount = point.groupBy('lat','lon').count()

In [ ]:
query = pointCount.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()